In [1]:
# Se importan las librerías necesarias.

import pandas as pd
import os, sys
from pymongo import MongoClient
import json

In [2]:
# Se crean las variables "dir", "path" y "sep" para facilitar el trabajo con rutas relativas.

dir = os.path.dirname
path = dir(os.getcwd())
sep = os.sep

path

'c:\\Users\\LeoFr\\Desktop\\BootCamp\\CARPETAS_profesorYalumno\\Reto_Tripulaciones'

In [3]:
# Se dispone de un csv con las diferentes estaciones de metro, y sus distintos accesos, clasificados por su condición de accesibilidad. Se cuenta además con las coordenadas de cada acceso.

accesos = pd.read_csv('..' + sep + 'data' + sep + 'accessos.csv')

In [4]:
accesos.head()

,FID,CODI_ACCES,NOM_ACCES,CODI_GRUP_ESTACIO,NOM_ESTACIO,ID_TIPUS_ACCESSIBILITAT,NOM_TIPUS_ACCESSIBILITAT,NUM_ASCENSORS,DATA,GEOMETRY
0,ACCESSOS.11101,11101,Residència sanitària,6660111,Hospital de Bellvitge,1,Accessible,1,2021-07-21,POINT (2.1066690928052068 41.34435669904031)
1,ACCESSOS.11201,11201,Rambla de la Marina / Avda. Granvia,6660112,Bellvitge,1,Accessible,1,2021-07-21,POINT (2.1115154528942424 41.35089549772364)
2,ACCESSOS.11202,11202,Rambla de la Marina / Merca 2,6660112,Bellvitge,3,No accessible,0,2021-07-21,POINT (2.1108418043280226 41.35075144720632)
3,ACCESSOS.11211,11211,Rambla de la Marina / Trav. Industrial,6660112,Bellvitge,3,No accessible,0,2021-07-21,POINT (2.110940199167465 41.351420324574185)
4,ACCESSOS.11212,11212,Rambla de la Marina / Mercat Municipal,6660112,Bellvitge,3,No accessible,0,2021-07-21,POINT (2.110483123470267 41.35123702229523)


In [5]:
# La cantidad de accesos clasificados por su tipo de accesibilidad es la siguiente: 

accesos.NOM_TIPUS_ACCESSIBILITAT.value_counts()

No accessible      177
Accessible         163
Semi-accessible      1
Name: NOM_TIPUS_ACCESSIBILITAT, dtype: int64

In [6]:
# En el dataset figuran 135 estaciones de metro diferentes.

accesos.NOM_ESTACIO.nunique()

135

In [7]:
# Se crea un dataframe con todos los ingresos accesibles de cada estación.

accesos_accesibles = accesos[accesos.NOM_TIPUS_ACCESSIBILITAT != 'No accessible']

In [8]:
accesos_accesibles

,FID,CODI_ACCES,NOM_ACCES,CODI_GRUP_ESTACIO,NOM_ESTACIO,ID_TIPUS_ACCESSIBILITAT,NOM_TIPUS_ACCESSIBILITAT,NUM_ASCENSORS,DATA,GEOMETRY
0,ACCESSOS.11101,11101,Residència sanitària,6660111,Hospital de Bellvitge,1,Accessible,1,2021-07-21,POINT (2.1066690928052068 41.34435669904031)
1,ACCESSOS.11201,11201,Rambla de la Marina / Avda. Granvia,6660112,Bellvitge,1,Accessible,1,2021-07-21,POINT (2.1115154528942424 41.35089549772364)
6,ACCESSOS.11311,11311,Avinguda Carrilet,6660113,Av. Carrilet,1,Accessible,1,2021-07-21,POINT (2.1026678046332026 41.35799997470539)
9,ACCESSOS.11403,11403,Renfe,6660114,Rambla Just Oliveras,1,Accessible,1,2021-07-21,POINT (2.1001537907418517 41.363993847124455)
11,ACCESSOS.11502,11502,Avinguda Can Serra,6660115,Can Serra,1,Accessible,1,2021-07-21,POINT (2.103393034797958 41.367609047748324)
...,...,...,...,...,...,...,...,...,...,...
336,ACCESSOS.113811,113811,Sant Feliu de Codines,6661138,Torre Baró | Vallbona,1,Accessible,1,2021-07-21,POINT (2.1792574126156485 41.45930587802702)
337,ACCESSOS.113901,113901,Avinguda Rasos de Peguera,6661139,Ciutat Meridiana,1,Accessible,1,2021-07-21,POINT (2.1746970655892848 41.46089542038299)
338,ACCESSOS.114001,114001,Circumval·lació,6661140,Can Cuiàs,1,Accessible,1,2021-07-21,POINT (2.1727425962893063 41.46267788387076)
339,ACCESSOS.114011,114011,Les Fustes,6661140,Can Cuiàs,1,Accessible,0,2021-07-21,POINT (2.173141755285573 41.46283124731911)


In [9]:
# Se crea una lista que contenga las coordenadas de cada uno de los ingresos accesibles.

geolocalizacion_accesos = []
for coordenadas in list(accesos_accesibles.GEOMETRY):
    geolocalizacion_accesos.append(coordenadas)

In [10]:
geolocalizacion_accesos[:3]

['POINT (2.1066690928052068 41.34435669904031)',
 'POINT (2.1115154528942424 41.35089549772364)',
 'POINT (2.1026678046332026 41.35799997470539)']

In [11]:
# Se separan las coordenadas en dos listas. Una correspondiente a longitudes y otra a latitudes.

acceso_longitudes = []
acceso_latitudes = []
for value in geolocalizacion_accesos:
    longitud = value.split()[1][1:]
    latitud = value.split()[2][:-1]
    acceso_longitudes.append(longitud)
    acceso_latitudes.append(latitud)

In [12]:
acceso_latitudes[:3]

['41.34435669904031', '41.35089549772364', '41.35799997470539']

In [13]:
acceso_longitudes[:3]

['2.1066690928052068', '2.1115154528942424', '2.1026678046332026']

In [14]:
# Se crea un dataframe con la totalidad de los ingresos accesibles de cada estación, junto con sus respectivas latitudes y longitudes.

df_accesos_accesibles = pd.DataFrame({'estacion': list(accesos_accesibles.NOM_ESTACIO), 'latitud': acceso_latitudes, 'longitud': acceso_longitudes})

In [15]:
df_accesos_accesibles

,estacion,latitud,longitud
0,Hospital de Bellvitge,41.34435669904031,2.1066690928052068
1,Bellvitge,41.35089549772364,2.1115154528942424
2,Av. Carrilet,41.35799997470539,2.1026678046332026
3,Rambla Just Oliveras,41.363993847124455,2.1001537907418517
4,Can Serra,41.367609047748324,2.103393034797958
...,...,...,...
159,Torre Baró | Vallbona,41.45930587802702,2.1792574126156485
160,Ciutat Meridiana,41.46089542038299,2.1746970655892848
161,Can Cuiàs,41.46267788387076,2.1727425962893063
162,Can Cuiàs,41.46283124731911,2.173141755285573


In [16]:
# Se guarda el dataframe como un archivo json.

df_accesos_accesibles.to_json('..' + sep + 'reports' + sep + 'accesibles_geo.json')

In [17]:
# Se lee el json como un diccionario.

with open('..' + sep + 'reports' + sep + 'accesibles_geo.json') as transportes:
    accesibles_dict = json.load(transportes)

In [18]:
# Se lee el archivo "password.json" como un diccionario. Dicho archivo se incluye en ".gitignore", evitando de ésta manera que se suba información sensible a GitHub.

with open('password.json') as password:
    json_password = json.load(password)

In [19]:
# Se realiza la conexión a MongoDB.

client =  MongoClient(json_password['enlace'])
db = client["turismo_accesible"]
collection = db["transportes"]

# Se inserta la colección en MongoDB.

collection.insert_one(accesibles_dict)